### Galilei's Falling Body Experiment

**A note on this document**
This document is known as a Jupyter notebook; it allows text and executable code to coexist in a very easy-to-read format. Blocks can contain text or executable code. For blocks containing code, press `Shift + Enter`, `Ctrl+Enter`, or click the arrow on the block to run the code. Earlier blocks of code need to be run for the later blocks of code to work.

Given the measurements, $y(t_0),y(t_1),\cdots,y(t_n)$, measured at $t_0,t_1,\cdots,t_n$, we want to find (estimate) $\mathbf{x} = [a \quad b \quad c]^\top$  that minimizes $||\mathbf{y} - \hat{\mathbf{y}}||^2$ where $\hat{y} = ax^2+bx+c$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

y0 = 60
v0 = 20
g = -9.8067


data = pd.read_csv("./data/falling_body.csv")
print(data)

We can convert the Data Frame data into numpy arrays:  

In [ ]:
t = data["time"].to_numpy()
y = data["y"].to_numpy()
print(t)
print(y)

Plot the data

In [ ]:
plt.figure(figsize=(5, 4))
plt.plot(t, y, "o")
plt.xlabel("time, sec")
plt.ylabel("height (y), m", rotation=90)
plt.grid(True)

Let's use `np.polyfit` to find the best polynomial function fit.
If our model is $\hat{y} = at + b$, we want to use a first-order polynomial function.

In [ ]:
coeffs = np.polyfit(t, y, 1)
print(coeffs)  # the first element is the slope and the second is the y-intercept.

The first element in the `coeffs` array represents the slope denoted as $a$, while the second element corresponds to the $y$-intercept labeled as $b$. Employing the `np.poly1d()` function enables us to generate a polynomial function with the coefficients of our preference. To illustrate:

```python
f = np.poly1d(c)
```

This code snippet generates a function $f = c_0x^n + c_1x^{n-1} + \cdots + c_{n-1}x + c_n$ with coefficients encapsulated within the vector $\mathbf{c}$.

In our specific scenario, using `np.poly1d(coeffs)` furnishes us with $y(t)$. Consequently, by supplying $t$ to the $y(t)$ function through `np.poly1d(coeffs)(t)`, denoted as `y(t) = np.poly1d(coeffs)(t)`, we achieve the desired output.

In [ ]:
y_hat = np.poly1d(coeffs)(t)

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot()

plt.plot(t, y, "o")  # plot the measured data
plt.plot(t, y_hat, "-")  # plot the fitted line
plt.xlabel("time, sec")
plt.ylabel("height (y), m", rotation=90)
plt.grid(True)

ax.text(2, 40, "M=1", style="italic")

The line doesn't align well with the measurements. Let's examine the discrepancies (residuals) and the root mean squared error (RMSE). The residual (error) is calculated as $y-\hat{y}$. The RMSE is then computed using the following formula:

$ RMSE = \sqrt{\frac{1}{n}\sum^{n}_{i=0}(y_i-\hat{y}_i)^2}$

In the realm of linear algebra, the norm of a vector is defined as:

$||\mathbf{v}|| = \sqrt{\sum^{n}_{i=0}v^2_i}$

Consequently, we can express the RMSE using the vector norm as:

$ RMSE = \frac{1}{\sqrt{n}}||\mathbf{v}|| $

Here is how we can find the RMSE:

In [ ]:
rmse = np.linalg.norm(y - y_hat) / np.sqrt(len(t))
print(rmse)

Let's now experiment with a quadratic function for our model, represented as $\hat{y} = at^2 + bt+c$

In [ ]:
coeffs = np.polyfit(t, y, 2) # 2nd order polynomial fit.
print(coeffs)  

In [ ]:
y_hat = np.poly1d(coeffs)(t)

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot()

plt.plot(t, y, "o")  # plot the measured data
plt.plot(t, y_hat, "-")  # plot the fitted line
plt.xlabel("time, sec")
plt.ylabel("height (y), m", rotation=90)
plt.grid(True)

ax.text(2, 40, "M=2", style="italic")

To find the RMSE

In [ ]:
rmse = np.linalg.norm(y - y_hat) / np.sqrt(len(t))
print(rmse)

The quadratic model exhibits a significantly lower RMSE compared to the first-order model.  Let's now try with a forth-order polynomial function.

In [ ]:
coeffs = np.polyfit(t, y, 4) # 4th order polynomial fit
print(coeffs) 


In [ ]:
y_hat = np.poly1d(coeffs)(t)

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot()

plt.plot(t, y, "o")  # plot the measured data
plt.plot(t, y_hat, "-")  # plot the fitted line
plt.xlabel("time, sec")
plt.ylabel("height (y), m", rotation=90)
plt.grid(True)

ax.text(2, 40, "M=4", style="italic")

In [ ]:
rmse = np.linalg.norm(y - y_hat) / np.sqrt(len(t))
print(rmse)

The fourth-order polynomial function demonstrates a slightly reduced RMSE in comparison to the second-order model. We'll delve deeper into fitting models of higher orders and address concerns related to overfitting.

Moving forward, let's address the problem using the least square estimate method. We need to formulate a system of linear equations as follow

$\begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}  = \begin{bmatrix} t^2_1 & t_1 & 1 \\ t^2_2 & t_2 & 1 \\ \vdots & \vdots & \vdots \\ t^2_n & t_n & 1 \end{bmatrix} \begin{bmatrix} a \\ b \\c \end{bmatrix}
$

or $\mathbf{y} = C\mathbf{q}$ where

$\mathbf{y} = [y_1 \quad y_2 \quad \cdots \quad y_n]^\top$,  $\mathbf{q} = [a \quad b \quad c]^\top$, and  $C = \begin{bmatrix} t^2_1 & t_1 & 1 \\ t^2_2 & t_2 & 1 \\ \vdots & \vdots & \vdots \\ t^2_n & t_n & 1 \end{bmatrix} $

We can formulate $C$ using `np.vstack()`. Make sure $C$ is an $n \times 3$ matrix, i.e., $C\in\mathbb{R}^{n\times 3}$  

In [ ]:
C = np.vstack([t**2, t, np.ones_like(t)]).T
print(C.shape)
print(C)

The psedu-inverse of the matrix, $C$ is given by $\tilde{C} = (C^{\top}C)^{-1}C^{\top}$.  In Python, we can use `np.linalg.pinv()` to find it.  The dimension should be 3 by $n$.

In [ ]:
pinvC = np.linalg.pinv(C)
print(pinvC.shape)
print(C)

We can now find $\mathbf{q} = [a \quad b \quad c]^\top$:

$\mathbf{q} = \tilde{C}\mathbf{y} = (C^{\top}C)^{-1}C^{\top}\mathbf{y}$

In [ ]:
q = pinvC @ y
print(q)

The values in xhat match the coefficients of the quadratic model obtained through the `np.polyfit` function.

In [ ]:
y_hat = np.poly1d(q)(t)

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot()

plt.plot(t, y, "o")  # plot the measured data
plt.plot(t, y_hat, "-")  # plot the fitted line
plt.xlabel("time, sec")
plt.ylabel("height (y), m", rotation=90)
plt.grid(True)

ax.text(2, 40, "M=2", style="italic")

Another approach is to utilize the `np.linalg.lstsq` function to compute the least square estimate.

In [ ]:
q = np.linalg.lstsq(C, y, rcond=None)[0]
print(q)

In [ ]:
help(np.linalg.lstsq)